# Probability weighted Autoencoder

In [ ]:
from river.compose import Pipeline
from river.preprocessing import MinMaxScaler

from river_torch.anomaly import ProbabilityWeightedAutoencoder
from river import metrics
from river.datasets import CreditCard
from torch import nn, manual_seed
import math

In [ ]:
_ = manual_seed(42)
dataset = CreditCard().take(5000)
metric = metrics.ROCAUC(n_thresholds=50)

def get_fc_ae(n_features):
    latent_dim = math.ceil(n_features / 2)
    return nn.Sequential(
        nn.Linear(n_features, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, n_features),
    )

ae = ProbabilityWeightedAutoencoder(build_fn=get_fc_ae, lr=0.01)
scaler = MinMaxScaler()
model = Pipeline(scaler, ae)

for x,y in dataset:
    score = model.score_one(x)
    metric.update(y_true=y, y_pred=score)
    model.learn_one(x=x)
print(f'ROCAUC: {metric.get():.4f}')